In [30]:
from getTexts import *
import pandas as pd 
import re 

In [31]:
csv_data = pd.read_csv('/Users/amycweng/Digital Humanities/sermons.csv')
tcpIDs = [ _ for _ in csv_data['id']]

In [32]:
def getTexts(folder,search):
    texts = {}
    for file in os.listdir(folder):
        if 'NOTES' not in file: 
            name = file.split('.')[0]
            if name in search: 
                path = os.path.join(folder,file)
                with open(path,'r') as file: 
                    data = file.readlines()
                if len(data) != 0: 
                    texts[name] = data[0]
    return texts

In [52]:
sermons = getTexts('/Users/amycweng/Digital Humanities/charityTXT',tcpIDs)
# sermons = getTexts('/Users/amycweng/Digital Humanities/charityTXT','A01523')
print(len(sermons))

70


In [56]:
words = ['charity','charitable','alm','bequest',
        'poor','rate','usury','usurer','bequeath',
        'money','beneficence','poverty','credit',
        'creditor','loan','lend','new','novel','novelty',
        'wealth','profit','profitable','beneficial',
        'commodity','thrift','thrifty','industry','bullion',
        'talon','talent','beggar','needy','bestow',
        'liberality','giving','penny','liberal',
        'shilling','score','restitution','default',
        'halfpenny','quarter','ounce','broker',
        'debtor','debt']
sus = []
for tcpID, text in sermons.items(): 
    if not re.search('charity|caritas|charitable|\balm\b|\balmes\b|poor|poor man|poverty|money|needy|hungry|beggar|taxation', text): 
        sus.append(tcpID)
print(len(sus), sus)

3 ['A01523', 'A01528', 'A10027']


In [ ]:
'''
Extract the entire body text from an XML document.  
'''
outputfolder = 'charityTXT'
convert(tcpIDs,outputfolder)
print(underscores)
convert(underscores,outputfolder)

'''
TCP charity texts that are not in EP: 
'B33867', 'B13871', 'B13868', 'B13878', 'B13862', 'B13860', 
'B09696', 'A86972', 'A19579', 'A09957'
'''

In [25]:
# patterns = ["Preach't at (.*?)[^\w\s\-]",'Preached at (.*?)[^\w\s\-]','preached at (.*?)[^\w\s\-]','preached in (.*?)[^\w\s\-\']']
patterns1 = ["preach't (.*?)$","preached at (.*?)$","preached in (.*?)$","preacht (.*?)$","before the (.*?)$", "before his (.*?)$"]
patterns2 = ["preacher at (.*?)$","preacher in (.*?)$","pastor at (.*?)$","pastor of (.*?)$"]
places = []
pastorPlaces = []
import csv

outFile = open(f'sermons_locations.csv','a+')
columns = ['id','title','author','date','location']
writer = csv.DictWriter(outFile, fieldnames=columns)
writer.writeheader()

located = []
for idx, title in enumerate(csv_data['title']):
    foundPattern1 = False
    for pattern in patterns1: 
        place = re.search(rf'{pattern}', title)
        if place: 
            # places.append(place.group())
            dict = {'id': tcpIDs[idx], 
                    'title': title,
                    'author': csv_data['author'][idx],
                    'date': csv_data['date'][idx],
                    'location': place.group()
                    }
            located.append(tcpIDs[idx])
            writer.writerow(dict)
            # print(f'{tcpIDs[idx]}: {place.group()}')
            foundPattern1 = True
            break
    if not foundPattern1: 
        for pattern in patterns2: 
            place = re.search(rf'{pattern}', title)
            if place: 
                dict = {'id': tcpIDs[idx], 
                    'title': title,
                    'author': csv_data['author'][idx],
                    'date': csv_data['date'][idx],
                    'location': place.group()
                }
                # pastorPlaces.append(place.group())
                located.append(tcpIDs[idx])
                writer.writerow(dict)
                foundPattern2 = True
                break

for idx, tcpID in enumerate(tcpIDs):
    if tcpID in located: 
        continue
    dict = {'id': tcpID, 
            'title': csv_data['title'][idx],
            'author': csv_data['author'][idx],
            'date': csv_data['date'][idx],
            'location': ''
            }
    writer.writerow(dict)

# print(len(places), len(pastorPlaces), len(places)+len(pastorPlaces))
# print(places)
# print(pastorPlaces)

In [69]:
def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

In [90]:
'''
Extracting title pages directly from TCP. 
Title pages often contain information about where the sermon was preached. 
'''
import csv 
folder = f'/Users/amycweng/Digital Humanities/'
outfile = open(f'{folder}/sermon_titles.csv','a+')
columns = ['id','title page']
writer = csv.DictWriter(outfile, fieldnames=columns)
writer.writeheader()

tcpIDs = ['A65610','A15015']
for tcpID in tcpIDs: 
    path = findTextTCP(tcpID)
    with open(path,'r') as file: 
        data = file.read()
    tag = SoupStrainer("div1", attrs={'type':'title page'})
    soup = BeautifulSoup(data,parse_only=tag,features='html.parser')
    title = []
    sentences = soup.find_all('p')
    for sentence in sentences: 
        title.append(sentence.text)
    if len(title) != 0: 
        title = ' '.join(title)
        title = re.sub('\n',' ',title)
        t = re.search(r'^(.*?) \d{4}',title)
        if t: 
            writer.writerow({'id': tcpID,  'title page': t.group()})
        else: 
            writer.writerow({'id': tcpID,  'title page': title})
outfile.close()